In [1]:
import pandas as pd
import numpy as np

In [2]:
# Importuojame funkcijas 
from functions import white_space
from functions import paskutinis_simbolis

In [3]:
# Sukuriame DF 
path1 = r'C:\Users\tvale\OneDrive - Girteka Logistics, UAB\Documents\02_mokymasis\final_poj\data\automobilis_data\automobilis\automobilis\automobilis.csv'
path2 = r'C:\Users\tvale\OneDrive - Girteka Logistics, UAB\Documents\02_mokymasis\final_poj\data\automobilis_data\automobilis\automobilis\spiders\automobilis2.csv'

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

In [4]:
# Sujungiame DF ir istriname dublikatus
automobilis_df = pd.concat([df1, df2]).drop_duplicates()

In [5]:
# Istriname nereikalingus stulpelius 
automobilis_df = automobilis_df.drop(['url', 'vardas', 'telefonas'], axis=1)

In [6]:
# Kaina. Paliekame tik skaities reiksmes
automobilis_df['kaina'] = automobilis_df['kaina'].str.replace('Kaina: ', '')
automobilis_df['kaina'] = automobilis_df['kaina'].str.replace(' EUR', '')
automobilis_df['kaina'] = white_space(automobilis_df['kaina']).astype(int)

In [7]:
# Metai. Paliekame tik metus 
automobilis_df['metai'] = automobilis_df['metai'].str[:4].astype(int)

In [8]:
# Bukle. Paliekame tik tas eilutes kuriose nera iraso. Be iraso automobilis be trukumu
automobilis_df = automobilis_df.loc[automobilis_df['bukle'].isna()]

In [9]:
# Galia. Padaliname stulpeli i dvi dalis. Isvalome duomenis, paliekame tik skaitines reiksmes.
# Pirma dalis - galia_kw
# Antra dalis - galia_ag
automobilis_df[['galia_kw', 'galia_ag']] = automobilis_df['galia'].str.split('(', expand=True)

automobilis_df['galia_kw'] = automobilis_df['galia_kw'].str.replace(' KW', '')
automobilis_df['galia_ag'] = automobilis_df['galia_ag'].str.replace(' AG)', '')

In [10]:
# Rida. Paliekame tik skaitines reiksmes.
automobilis_df['rida'] = automobilis_df['rida'].str.replace(' km', '')

In [11]:
# Pavaru deze. Standartizuojame pavadinimus, kad butu tik Mechanine ir Automatine 
automobilis_df['pavaru_deze'] = np.where(
    (automobilis_df['pavaru_deze'] == 'Pusiau automatinė'), 'Automatinė', automobilis_df['pavaru_deze']
)

In [12]:
# Vairo padetis. Paliekame tik tas reiksmes kur yra Kairėje
automobilis_df = automobilis_df.loc[automobilis_df['vairo_padetis'] == 'Kairėje']

In [13]:
# Darbinis turis. Daliname stulpeli i dvi dalis. Paliekame tik skaitines reiksmes.
# Pirma dalis - variklio_turis_ltr
# Antra dalis - variklio_turis_cm3
automobilis_df[['variklio_turis_ltr', 'variklio_turis_cm3']] = automobilis_df['darbinis_turis'].str.split('(', expand=True)

automobilis_df['variklio_turis_ltr'] = automobilis_df['variklio_turis_ltr'].str.replace(' litr.', '')
automobilis_df['variklio_turis_cm3'] = automobilis_df['variklio_turis_cm3'].str.replace(' cm³)', '')

In [14]:
# Pavadinimas. Pavadinimas stulpeli padaliname i gamintojas ir modelis
automobilis_df[['pavadinimas1', 'pavadinimas2']] = automobilis_df['pavadinimas'].str.split(' m.,', expand=True)
automobilis_df['pavadinimas1'] = automobilis_df['pavadinimas1'].str[: -5]

# I atskirus stulpelius isdaliname pavadinima
automobilis_df['gamintojas'] = automobilis_df['pavadinimas1'].str.split().str.get(0)
automobilis_df['modelis'] = automobilis_df['pavadinimas1'].str.split().str.get(1)
automobilis_df['modelis2'] = automobilis_df['pavadinimas1'].str.split().str.get(2)

# Apjungeme modelio pavadinima
automobilis_df['modelis_final'] = automobilis_df['modelis'] + automobilis_df['modelis2']

In [15]:
automobilis_df['modelis_final2'] = np.where(
    (automobilis_df['modelis_final'].isna()), automobilis_df['modelis'], automobilis_df['modelis_final']
)

In [16]:
# Istriname nereikalingus stulpelius 
automobilis_df = automobilis_df.drop([
    'darbinis_turis', 
    'galia', 
    'bukle', 
    'pavadinimas2', 
    'pavadinimas', 
    'vairo_padetis', 
    'pavadinimas1',
    'modelis',
    'modelis2',
    'modelis_final',
    'variklio_turis_cm3'
    ], axis=1
)

In [17]:
automobilis_df = automobilis_df.reset_index(drop=True)

In [18]:
automobilis_df = automobilis_df.rename(columns={
        'kebulo_tipas': 'kebulas',
        'variklio_turis_ltr': 'variklio_turis',
        'modelis_final2': 'modelis'
    }
)

In [19]:
automobilis_df

,kaina,metai,kebulas,kuras,duru_sk,pavaru_deze,rida,galia_kw,galia_ag,variklio_turis,gamintojas,modelis
0,8690,2013,Universalas,Dyzelinas,4/5,Mechaninė,232000,103,138,2,Volkswagen,Passat
1,29,2003,Vienatūris,Dyzelinas,NaN,NaN,NaN,NaN,NaN,NaN,Mercedes-Benz,A170
2,10799,2014,Hečbekas,Benzinas,4/5,Automatinė,219874,97,130,1600,Toyota,Auris
3,7777,2013,Vienatūris,Dyzelinas,4/5,Mechaninė,213000,100,134,1.6,Opel,Zafira
4,8000,2007,Visureigis,Dyzelinas,4/5,Automatinė,NaN,173,231,3,BMW,X5
...,...,...,...,...,...,...,...,...,...,...,...,...
1038,3200,2007,Hečbekas,Dyzelinas,4/5,Automatinė,147472,60,80,2,Mercedes-Benz,A160
1039,1000,1997,Universalas,Dyzelinas,4/5,Mechaninė,NaN,103,138,25,Audi,A6
1040,3850,2010,Hečbekas,Dyzelinas,4/5,Mechaninė,111000,66,88,NaN,Toyota,Yaris
1041,3300,2006,Universalas,Dyzelinas,4/5,Automatinė,270600,136,182,2.4,Volvo,XC70
